# Compiler

Julia 編譯器被設計為可以提供編譯過程中間碼，透過中間碼我們可以了解到編譯器最佳化程式碼的過程，以及型別穩定性（type stablility）在編譯過程中所扮演的角色。這邊我們有個範例，是將輸入的數值平方的函式：

In [1]:
function square(a)
    a^2
end

square (generic function with 1 method)

In [2]:
square(2)

4

## Parsing

在編譯過程的第一步是解析表示式（expression），解析（parsing）的過程是將原始碼解析成電腦能讀懂的結構。一般電腦能夠讀懂的結構會以樹狀結構來表示，稱為抽象語法樹（abstract syntax tree），簡稱 AST。

In [3]:
program = "square(2)"

"square(2)"

AST 是一種資料結構，是由原始碼（字串）透過解析後所建構的而成的。這種樹狀結構是用來表示語言的表達式本身，與表達式等價。

In [4]:
expression = Meta.parse(program)

:(square(2))

In [5]:
typeof(expression)

Expr

在 Julia 中便是以 `Expr` 結構來表示，並可以用 `dump` 來呈現結構。

In [6]:
dump(expression)

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol square
    2: Int64 2


`Expr` 當中紀錄了這個表達式的行為是一個函式呼叫（`:call`），資訊會紀錄在 `head` 中。

In [7]:
expression.head

:call

函式呼叫需要兩個參數（`args`），第一個參數為呼叫的函式名稱 `:square`，第二個參數為函式的參數 `2`。

In [8]:
expression.args

2-element Array{Any,1}:
  :square
 2       

可以試試看稍微複雜一點的表達式。

In [9]:
expr = Meta.parse("2^2 + 5*3 + 1")
dump(expr)

Expr
  head: Symbol call
  args: Array{Any}((4,))
    1: Symbol +
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol ^
        2: Int64 2
        3: Int64 2
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Int64 5
        3: Int64 3
    4: Int64 1


## Julia IR

解析過的 `Expr` 會進一步被轉換成 Julia IR，這是一種中間碼（intermediate representation），它會在 Julia 語言當中被處理。可以透過 `@code_lowered` 來看到轉換過後的結果。由於 Julia 編譯器是一種即時（just-in-time）編譯器，程式碼的編譯只有在被呼叫的時候才會被執行，我們需要明確給予值來執行程式碼，才能看到 Julia IR。

In [10]:
@code_lowered square(1)

CodeInfo(
1 ─ %1 = Core.apply_type(Base.Val, 2)
│   %2 = (%1)()
│   %3 = Base.literal_pow(Main.:^, a, %2)
└──      return %3
)

接著，Julia 編譯器會去蒐集型別的資訊以及做型別推論（type inference）來填補 Julia IR 中的型別資訊。Typed Julia IR 可以由 `@code_typed` 得到。

In [11]:
@code_typed square(1)

CodeInfo(
1 ─ %1 = Base.mul_int(a, a)::Int64
└──      return %1
) => Int64

這邊我們可以看到它仍保留有函式中運算的資訊。有型別資訊，有運算的過程及回傳值，中間運算過程所產生的變數會以 `%1` 的方式紀錄下來，在後續會被轉換成暫存器位址。

In [12]:
@code_warntype square(1)

Variables
  #self#::Core.Compiler.Const(square, false)
  a::Int64

Body::Int64
1 ─ %1 = Core.apply_type(Base.Val, 2)::Core.Compiler.Const(Val{2}, false)
│   %2 = (%1)()::Core.Compiler.Const(Val{2}(), false)
│   %3 = Base.literal_pow(Main.:^, a, %2)::Int64
└──      return %3


`@code_warntype` 則可以在轉換成 LLVM IR 之前警告哪一部份的程式碼的型別資訊是不充足的。

## LLVM IR

Julia 中間會透過 LLVM 作為中介，並在 LLVM 進行最佳化，最後並產生組合語言。LLVM IR 是 LLVM 的中間碼，我們可以測試兩種不同型別的參數進到函式中，會編譯成兩種不同的 LLVM IR，可以透過 `@code_llvm` 得到。

In [13]:
@code_llvm square(1)


;  @ In[1]:2 within `square'
define i64 @julia_square_17434(i64) {
top:
; ┌ @ intfuncs.jl:244 within `literal_pow'
; │┌ @ int.jl:54 within `*'
    %1 = mul i64 %0, %0
; └└
  ret i64 %1
}


In [14]:
@code_llvm square(1.)


;  @ In[1]:2 within `square'
define double @julia_square_17844(double) {
top:
; ┌ @ intfuncs.jl:244 within `literal_pow'
; │┌ @ float.jl:405 within `*'
    %1 = fmul double %0, %0
; └└
  ret double %1
}


雖然數值上是一樣的，但是在底層的 LLVM IR 部份是截然不同的，會執行不同的最佳化。

## Assembly

最後，經過一番最佳化後，會由機器碼生成（machine code generation）轉換成組合語言（assembly），可以由 `@code_native` 得到。

In [15]:
@code_native square(1)

	.text
; ┌ @ In[1]:2 within `square'
; │┌ @ intfuncs.jl:244 within `literal_pow'
; ││┌ @ In[1]:2 within `*'
	imulq	%rdi, %rdi
; │└└
	movq	%rdi, %rax
	retq
	nopl	(%rax,%rax)
; └


In [16]:
@code_native square(1.)

	.text
; ┌ @ In[1]:2 within `square'
; │┌ @ intfuncs.jl:244 within `literal_pow'
; ││┌ @ In[1]:2 within `*'
	vmulsd	%xmm0, %xmm0, %xmm0
; │└└
	retq
	nopw	%cs:(%rax,%rax)
; └


## Compiler and function

到最後編譯器都會將程式碼轉換成一條一條的組合語言，組合語言是由指令所組成，一條指令就相當於一個動作。在程式碼當中，我們也相對應看重函式的行為與互動。函式的參數型別也就扮演相當重要的角色，參數型別提供了編譯過程中型別的重要資訊。有了型別，LLVM IR 可以有充足的型別資訊，編譯器也可以對型別做最佳化，產生相當有效率的機器碼。反之，如果提供的型別資訊不足，取而代之，就需要相當多的程式碼在執行期間檢查這些型別資訊，如此會讓程式的執行變緩慢。

如果我們將函式的參數型別限定為特定型別，這樣這個函式只能適用這個特定型別。

In [17]:
function square(a::Int32)
    a^2
end

square (generic function with 2 methods)

雖然它產生的 LLVM IR 是針對特定型別最佳化的，有充足的型別資訊，但是彈性很差。

In [18]:
@code_llvm square(Int32(1))


;  @ In[17]:2 within `square'
define i32 @julia_square_17851(i32) {
top:
; ┌ @ intfuncs.jl:244 within `literal_pow'
; │┌ @ int.jl:54 within `*'
    %1 = mul i32 %0, %0
; └└
  ret i32 %1
}


因此，設計函式的時候我們就需要考慮參數型別。一個好的演算法被實作，會被化成一個函式，而函式的參數型別是夠廣義的，所以在 Julia 中設計的都是所謂的 generic function。Generic function 常常考慮的是實作一個夠廣義的演算法，而較少依賴型別，但並非完全不被型別所限制，有效能最佳化的必要，我們還是得取捨，去拆分函式或是針對型別做設計。要避免針對特定型別的實作，我們需要將型別抽象化，使用抽象型別，讓某一類抽象型別下的具體型別都可以適用這個實作。

In [19]:
function square(a::Number)
    a^2
end

square (generic function with 3 methods)

考慮可以取平方的型別來說，`Number` 是最合適的範圍，在這個範圍內的值都是可以取平方的。因此，Julia 編譯器就可以在執行期間針對不同的參數，即時編譯出相對應而且有效率的機器碼了。

In [20]:
@code_llvm square(1)


;  @ In[19]:2 within `square'
define i64 @julia_square_17852(i64) {
top:
; ┌ @ intfuncs.jl:244 within `literal_pow'
; │┌ @ int.jl:54 within `*'
    %1 = mul i64 %0, %0
; └└
  ret i64 %1
}


In [21]:
@code_llvm square(1.)


;  @ In[19]:2 within `square'
define double @julia_square_17853(double) {
top:
; ┌ @ intfuncs.jl:244 within `literal_pow'
; │┌ @ float.jl:405 within `*'
    %1 = fmul double %0, %0
; └└
  ret double %1
}


In [22]:
@code_llvm square(UInt8(1))


;  @ In[19]:2 within `square'
define i8 @julia_square_17855(i8) {
top:
; ┌ @ intfuncs.jl:244 within `literal_pow'
; │┌ @ int.jl:54 within `*'
    %1 = mul i8 %0, %0
; └└
  ret i8 %1
}


`methods` 可以進一步查詢目前有宣告的實作版本有哪些。

In [23]:
methods(square)

# 3 methods for generic function "square":
[1] square(a::Int32) in Main at In[17]:2
[2] square(a::Number) in Main at In[19]:2
[3] square(a) in Main at In[1]:2